# Uses interaction to push triples to the brain

Before running, start GraphDB and make sure that there is a sandbox repository

In [1]:
# general imports for EMISSOR and the BRAIN
import emissor as em
from cltl import brain
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType
#from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.reply_generation.lenka_replier import LenkaReplier

import os
import time
import uuid
from datetime import datetime
from datetime import date
import requests


[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import sys
import os
src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util
import intentions.talk as talk

In [3]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "../../data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-01-16:42:40  Created 
Directory  ../../data/2021-11-01-16:42:40/image  Created 


In [4]:
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)
replier = LenkaReplier()

<class 'pathlib.PosixPath'>
2021-11-01 16:42:41,886 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-11-01 16:42:41,886 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Clearing brain
2021-11-01 16:42:42,295 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Checking if ontology is in brain
2021-11-01 16:42:42,297 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-11-01 16:42:42,996 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-01 16:42:44,458 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-11-01 16:42:44,459 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-11-01 16:42:44,461 -    DEBUG -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-11-01 16:42:44,464 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-11-01 16:42:44,546 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query
2021-11-01 16:42:44,550 -     INF

In [6]:
#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = "Hi there. Who are you " + HUMAN_NAME + "?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = input('\n')
print(HUMAN_NAME + ": " + utterance)
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)
    
    thoughts = talk.process_text_and_reply (scenario, place_id, location, HUMAN_ID, textSignal, replier, my_brain)
    
    print(AGENT + ": " + thoughts)
    textSignal = d_util.create_text_signal(scenario, thoughts)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')


Leolani2: Hi there. Who are you Stranger?



 I like cats


Stranger: I like cats
2021-11-01 16:54:34,246 -     INFO - cltl.triple_extraction.api.Chat (stranger)          000 - << Start of Chat with stranger >>
2021-11-01 16:54:35,246 -     INFO - cltl.triple_extraction.api.Chat (stranger)          001 -   stranger: "I like cats"
{'subject': {'label': 'stranger', 'type': ['noun.person']}, 'predicate': {'label': 'like', 'type': ['verb.emotion']}, 'object': {'label': 'cats', 'type': ['noun.animal']}}
{'subject': {'label': 'stranger', 'type': 'person'}, 'predicate': {'label': 'like', 'type': 'emotion'}, 'object': {'label': 'cats', 'type': 'animal'}}
{'chat': '2021-11-01-16:42:40', 'turn': 'e70c69fa-8cda-48df-9522-e731aa510009', 'author': 'stranger', 'utterance': 'I like cats', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'position': '0-11', 'subject': {'label': 'stranger', 'type': 'person'}, 'predicate': {'label': 'like', 'type': 'emotion'}, 'object': {'label': 'cats', 'type': 'animal'}, 'perspective': {'sentiment': '0.75', 'certainty': 1, 'pol

AttributeError: 'dict' object has no attribute 'triple'

In [6]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook